In [4]:
import pandas as pd
import seaborn as sns
import numpy as np
import matplotlib.pyplot as plt
import datetime as dt
from ipywidgets import interact, interactive, fixed, interact_manual
import ipywidgets as widgets
from IPython.display import HTML

In [5]:
HTML("""\
<style>
.app-subtitle {
    font-size: 1.5em;
    border-bottom: 2px solid rgb(148,39,44);
}

.app-subtitle a {
    color: #106ba3;
}

.app-subtitle a:hover {
    text-decoration: underline;
}

.app-sidebar p {
    margin-bottom: 1em;
    line-height: 1.7;
}

.app-sidebar a {
    color: #106ba3;
}

.app-sidebar a:hover {
    text-decoration: underline;
}

.logo {
    max-width: 50%;
    height: auto;
    display: block;
    margin-left: auto;
    margin-right: auto;
}

.top {
    border-bottom: 5px solid rgb(148,39,44);
}

.focus {
    font-size: xlarger;
}
</style>
""")

In [1]:
class App:
    def __init__(self):
        self.set_general_config()
        self.load_data()
        self.create_container()
        self._update_app()
    
    def set_general_config(self):
        self._width = 12
        self._height = 6
        sns.set(rc = {'figure.figsize':(self._width,self._height)})
        sns.set(font_scale=1.5)
        sns.set_style("whitegrid")
    
    def load_data(self):
        self.df_nao_resp = pd.read_csv('web_sai/QuestionariosNaoRespondidos_Alunos.csv', sep=',')
        self.data_prepare()
        
    def data_prepare(self):
        pd.set_option('display.max_rows', self.df_nao_resp.shape[0]+1)
        self.df_nao_resp['Semestre de Ingresso'] = self.df_nao_resp.loc[:, 'Semestre de Ingresso'].apply(lambda v: v.split(':')[1])

    #####
    # Q:Tem previsão para tomar a 2a. dose?
    ####
    def _create_plot_q5_perc(self):
        self.plot_percentual_question('Tem previsão para tomar a 2a. dose?')
    
    def _create_plot_q5_curso(self):
        self.plot_count_questao(questao = 'Tem previsão para tomar a 2a. dose?')
    
    def _create_plot_q5_turno(self):
        self.plot_count_questao(questao = 'Tem previsão para tomar a 2a. dose?', y='Turno')

    #####
    # Q: Imunização pra cada disciplina/curso
    #####
    def _create_plot_imunizacao(self, curso):
        self._plot_disciplina_perc('Imunização',curso)
    
    def change_DrpCourse(self, drp_chg):
        self._plot_container_imuni_course.clear_output(wait=True)
        with self._plot_container_imuni_course:
            self. _create_plot_imunizacao(self.drp_course.value)
            plt.show()

    def create_container(self):
        self.drp_course = widgets.Dropdown(
            options=np.insert(self.df_nao_resp.loc[:, 'Curso'].unique(), 0, 'Todos', axis=0),
            description='Curso:',
            disabled=False
        )
        self.drp_semester = widgets.Dropdown(
            options=np.insert(self.df_nao_resp.loc[:, 'Semestre de Ingresso'].unique(), 0, 'Todos', axis=0),
            description='Ingresso:',
            disabled=False
        )
        self.drp_course.observe(self.change_DrpCourse)
        self.drp_semester.observe(self.change_DrpSemester)
        self._listagem = widgets.Output()
        self._total = widgets.Output()
        self.container = widgets.VBox([
            widgets.HTML(
                (
                    '<div class="top"><img class="logo" src="images/logo_fatecrp_cps_colorido_fundobranco.png"/></div>'
                    '<h1>WebSAI</h1>'
                    '<h2 class="app-subtitle"><a href="www.fatecrp.edu.br">fatecrp.edu.br</a></h2>'
                ), layout=widgets.Layout(margin='0 0 0 0')
            ),
            widgets.VBox([
                widgets.HTML('<h2>Estudantes que Não Responderam ao Questionário</h2>', layout=widgets.Layout(margin='0 0 0 0')),
                widgets.HBox([self.drp_course, self.drp_semester], layout=widgets.Layout(margin='0 0 0 0')),
                self._listagem
            ])
        ], layout=widgets.Layout(flex='1 1 auto', margin='0 auto 0 auto', max_width='1024px'))

    def change_DrpCourse(self, ev):
        self._listar_estudantes(self.drp_course.value, self.drp_semester.value) 
        
    def change_DrpSemester(self, ev):
        self._listar_estudantes(self.drp_course.value, self.drp_semester.value) 

    def _on_change(self, _):
        self._update_app()
        
    def _update_app(self):            
        self._listar_estudantes()

    def _listar_estudantes(self, curso='Todos', semestre='Todos'):
        _df = self.df_nao_resp
        if curso != 'Todos':
            _df = _df.loc[_df.loc[:, 'Curso'] == curso]
        if semestre != 'Todos':
            _df = _df.loc[_df.loc[:, 'Semestre de Ingresso'] == semestre]
        self._listagem.clear_output()
        with self._listagem:
            display(_df.loc[:, ['Nome', 'Semestre de Ingresso', 'Curso']])        

In [17]:
app = App()
app.container